In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import collections
from matplotlib.colors import ListedColormap
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer  ## 방법 2개
from collections import Counter
from gensim.models import Word2Vec



plt.rc('font', family='AppleGothic') 
plt.rcParams['axes.unicode_minus'] =False

In [7]:
product = pd.read_csv("mecab-ko-dic-2.1.1-20180720/Zigzag_products.csv")
product

,순위,카테고리,세부 카테고리,브랜드,상품명,정가,할인율,할인가,최저가,평균 리뷰 점수,리뷰 개수,배송,링크,상품ID
0,1,의류,상의,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,43200,0.31,37300.0,29840,4.8,4509,일반배송무료배송,https://zigzag.kr/catalog/products/130108350,130108350
1,2,의류,상의,리얼코코,리얼코코 [단독] [봄신상🌸/브이넥&amp;라운드넥/made] 시오 니트 - 20 ...,51000,0.57,24630.0,22160,4.8,16998,직진배송,https://zigzag.kr/catalog/products/147921597,147921597
2,3,의류,하의,고고싱,고고싱 [역대급53만장🏆11컬러/부츠컷슬랙스🌷봄팬츠] 반하이 모찌슬랙스 #세미부츠컷...,34900,0.36,24800.0,22320,4.7,77846,직진배송,https://zigzag.kr/catalog/products/113041967,113041967
3,4,의류,상의,데일리쥬,데일리쥬 [단독][1만장판매/MADE] 라스카 레이어드 텐셀 라운드 티셔츠 (워머옵...,30000,0.36,21370.0,19230,4.7,5248,직진배송,https://zigzag.kr/catalog/products/128943162,128943162
4,5,의류,상의,마이온샵,마이온샵 [BEST][누적2만장돌파🥇][4타입] #MADE 메로트 골지 니트티 (4...,43000,0.42,31160.0,24920,4.7,3215,직진배송,https://zigzag.kr/catalog/products/129147932,129147932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,46,뷰티,NaN,약손명가헬스케어,약손명가헬스케어 [직잭픽][원조리프팅밴드] 약손명가 작은얼굴관리 리프팅 밴드 1Bo...,69000,0.47,36900.0,36900,4.8,1315,무료배송,https://zigzag.kr/catalog/products/132075437,132075437
296,47,뷰티,NaN,바닐라코,바닐라코 스머징 립 펜슬,14000,0.28,10080.0,10080,4.8,1868,일반배송무료배송,https://zigzag.kr/catalog/products/125973586,125973586
297,48,뷰티,NaN,투쿨포스쿨,투쿨포스쿨 💖코코초PICK/NEW컬러💖 스머징 트임라이너,12000,0.18,9900.0,9900,4.8,8127,무료배송,https://zigzag.kr/catalog/products/132867464,132867464
298,49,뷰티,NaN,베노프,베노프 [15회분] 20g 단백질쉐이크 초콜릿시리얼/미숫가루시리얼/초코헤이즐넛라떼 ...,39900,0.32,32900.0,26970,4.9,3940,무료배송,https://zigzag.kr/catalog/products/140368165,140368165


In [4]:
review = pd.read_csv("review_전처리.csv")
review.head()

/var/folders/4z/77ppy0w97mq12l8j00pcdgr80000gn/T/ipykernel_86131/3083856169.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12,13,14,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  review = pd.read_csv("review_전처리.csv")


,리뷰순서,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,...,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID,review_count,review_tfidf,keyword,clean_count,clean_tfidf
0,1,의류,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,솔링이,25.02.01,-3kg블랙,정사이즈예요,아주 만족해요,화면과 비슷해요,...,5.0,[158/42kg) 이 옷이 유독 슬림해보이는 거 같아요 ㅎㅎ 여리여리한 느낌이 ...,직진,있음,130108350.0,['15842kg' '15842kg 옷이' '같아요' '같아요 여리여리한' '나고'...,['15842kg' '15842kg 옷이' '같아요' '같아요 여리여리한' '나고'...,"['어깨', '브이넥이라 어깨', '어깨 라인이', '15842kg', '15842...",15842kg 15842kg 옷이 같아요 같아요 여리여리한 나고 나고 브이넥이라 나...,15842kg 15842kg 옷이 같아요 같아요 여리여리한 나고 나고 브이넥이라 나...
1,2,의류,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,콩콩땅,25.01.30,발렛베이지,정사이즈예요,아주 만족해요,화면과 비슷해요,...,5.0,"⭐️ 스펙 : 158cm/50kg , 상체 보다 하체 발달형 ⭐️ 핏, 사이즈 ...",직진,있음,130108350.0,['158cm50kg' '158cm50kg 상체' '감싸줘요' '감싸줘요 재질이' ...,['158cm50kg' '158cm50kg 상체' '감싸줘요' '감싸줘요 재질이' ...,"['그자체 어깨가', '어깨가 넓은', '158cm50kg', '158cm50kg ...",158cm50kg 158cm50kg 상체 감싸줘요 감싸줘요 재질이 그자체 그자체 어...,158cm50kg 158cm50kg 상체 감싸줘요 감싸줘요 재질이 그자체 그자체 어...
2,3,의류,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,미소세계,25.01.26,발렛베이지,정사이즈예요,아주 만족해요,화면과 비슷해요,...,5.0,발레코어룩의 정석인 아이템 발견했네여!!!ㅎㅎ 완전 여리여리해보이는 효과가 있어욤ㅠ...,일반,있음,130108350.0,['가격에' '가격에 퀄리티면' '가끔씩' '가끔씩 여리해보이는' '너무' '너무 ...,['가격에' '가격에 퀄리티면' '가끔씩' '가끔씩 여리해보이는' '너무' '너무 ...,"['입어봤는데 너무', '옷을 입고더보기지그재그', '처음 입어봤는데', '컬러별로...",가격에 가격에 퀄리티면 가끔씩 가끔씩 여리해보이는 너무 너무 예쁜데요 리뷰입니다 무...,가격에 가격에 퀄리티면 가끔씩 가끔씩 여리해보이는 너무 너무 예쁜데요 리뷰입니다 무...
3,4,의류,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,도귱,25.01.24,발렛베이지,생각보다 커요,보통이에요,화면과 비슷해요,...,5.0,🩷핏 너무 이뻐요!! 예전에 이런 디자인의 티셔츠 구매했다가 저한테 너무 크고 주름...,직진,있음,130108350.0,['같아요더보기지그재그' '같아요더보기지그재그 체험단' '구매했다가' '구매했다가 ...,['같아요더보기지그재그' '같아요더보기지그재그 체험단' '구매했다가' '구매했다가 ...,"['구매했다가', '구매했다가 저한테', '티셔츠', '디자인의 티셔츠', '티셔츠...",같아요더보기지그재그 같아요더보기지그재그 체험단 구매했다가 구매했다가 저한테 너무 너...,같아요더보기지그재그 같아요더보기지그재그 체험단 구매했다가 구매했다가 저한테 너무 너...
4,5,의류,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,벚나무,25.01.23,발렛베이지,정사이즈예요,아주 만족해요,화면과 비슷해요,...,5.0,💜169cm/51kg/235/75D 최근 산 상의중에 제일 맘에 듭니다 ✔️일단...,직진,있음,130108350.0,['169cm51kg23575d' '169cm51kg23575d 최근' '가능한데'...,['169cm51kg23575d' '169cm51kg23575d 최근' '가능한데'...,"['선택', '워머스타일 좋아해서', '169cm51kg23575d', '워머 선택...",169cm51kg23575d 169cm51kg23575d 최근 가능한데 가능한데 갠...,169cm51kg23575d 169cm51kg23575d 최근 가능한데 가능한데 갠...


#### 상품 : product130108350

In [5]:
product130108350 = review[review.상품ID == 130108350]
product130108350 = product130108350[['상품명', '리뷰 코멘트', 'clean_count', 'clean_tfidf']]
product130108350.head(1)

,상품명,리뷰 코멘트,clean_count,clean_tfidf
0,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,[158/42kg) 이 옷이 유독 슬림해보이는 거 같아요 ㅎㅎ 여리여리한 느낌이 ...,15842kg 15842kg 옷이 같아요 같아요 여리여리한 나고 나고 브이넥이라 나...,15842kg 15842kg 옷이 같아요 같아요 여리여리한 나고 나고 브이넥이라 나...


In [75]:
product130108350['상품명'][0]

'퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작) 로잔나 여리핏 랩 실루엣 워머탑 티 - 10color'

📌 필요 함수 정의

In [73]:
# Headline 형태소 분석 (명사, 동사, 형용사)
from konlpy.tag import Okt
def topKeywords(okt,text):
    noun = []
    adj = []
    for word, pos in okt.pos(text, stem=True):
        if pos == 'Noun':
            noun.append(word)
        elif pos == 'Adjective':
            adj.append(word)           
    return noun, adj

In [52]:
def 빈도분석(df, 품사):
    # adj 열 추출 및 단어 분리
    adj_list = df[품사].str.split().explode().tolist()

    # 빈도 계산
    word_counts = Counter(adj_list)

    # 상위 10개 형용사 출력
    top_adjectives = word_counts.most_common(10)
    print("가장 많이 언급된 품사 (빈도순):")
    for word, count in top_adjectives:
        print(f"{word}: {count}")

In [53]:
def PMI(df, 품사):
    # 단어와 문서별 출현 행렬 생성
    adj_words = df[품사].str.split().explode().unique()
    doc_word_counts = {word: [] for word in adj_words}
    for idx, row in df.iterrows():
        words = set(row[품사].split())
        for word in adj_words:
            doc_word_counts[word].append(1 if word in words else 0)

    # PMI 계산
    total_docs = len(df)
    word_freq = Counter(df[품사].str.split().explode())
    pmi_scores = {}
    for word in adj_words:
        p_w = word_freq[word] / sum(word_freq.values())  # 단어 확률
        for doc_idx in range(total_docs):
            p_d = 1 / total_docs  # 문서 확률 (균등 가정)
            p_w_d = doc_word_counts[word][doc_idx]  # 단어-문서 동시 출현
            if p_w_d > 0:
                pmi = np.log(p_w_d / (p_w * p_d))
                pmi_scores[(word, doc_idx)] = pmi

    # 상위 PMI 단어 추출
    top_pmi = sorted(pmi_scores.items(), key=lambda x: x[1], reverse=True)[:10]
    print("PMI 상위 품사:")
    for (word, doc_idx), score in top_pmi:
        print(f"{word} (문서 {doc_idx}): {score}")

In [54]:
def WordEmbedding(df, 품사):
    # 문장 리스트 생성
    sentences = df[품사].str.split().tolist()

    # Word2Vec 모델 학습
    model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

    # 상위 빈도 단어
    word_counts = Counter(df[품사].str.split().explode())
    top_words = [word for word, count in word_counts.most_common(5)]

    print("상위단어")
    print(top_words)
    print()

    # 유사 단어 추출
    print("상위 단어와 유사한 품사:")
    for word in top_words:
        similar = model.wv.most_similar(word, topn=3)
        print(f"{word}: {similar}")

##### 1. 상품명

In [ ]:
okt = Okt()
df = pd.DataFrame(None, columns=['noun','adj','all'])
df.loc[0] = [1,2,3]
for idx,text in enumerate(product130108350['상품명']):
    n,a = topKeywords(okt,text)
    df.loc[idx] = [' '.join(n), ' '.join(a), ' '.join(n)+' '+' '+' '.join(a)]

tfidf = TfidfVectorizer()
res = tfidf.fit_transform(df['all']).toarray()

# TF-IDF 기준 상위 n_top 개수 키워드 추출
n_top = 300
importance = np.argsort(np.asarray(res.sum(axis=0)).ravel())[::-1]
tfidf_feature_names = np.array(tfidf.get_feature_names_out())
# print(tfidf_feature_names[importance[:n_top]])


# TF-IDF 기준 상위 top 개수만큼 품사별 키워드 추출 (명사, 동사, 형용사)
noun,adj = topKeywords(okt,' '.join(tfidf_feature_names[importance[:n_top]]))

noun_top = 100
print(f'명사 top : {noun[:noun_top]}')

명사 top : ['평점', '퍼빗', '제작', '워머탑', '여리핏', '실루엣', '색소', '모델', '로잔', '랩티', '단독', '누적', '기모', '극찬']

형용사 top : []


##### 2. 리뷰 코멘트

In [63]:
okt = Okt()
df = pd.DataFrame(None, columns=['noun','adj','all'])
df.loc[0] = [1,2,3]
for idx,text in enumerate(product130108350['리뷰 코멘트']):
    n,a = topKeywords(okt,text)
    df.loc[idx] = [' '.join(n), ' '.join(a), ' '.join(n)+' '+' '+' '.join(a)]

tfidf = TfidfVectorizer()
res = tfidf.fit_transform(df['all']).toarray()

# TF-IDF 기준 상위 n_top 개수 키워드 추출
n_top = 300
importance = np.argsort(np.asarray(res.sum(axis=0)).ravel())[::-1]
tfidf_feature_names = np.array(tfidf.get_feature_names_out())
# print(tfidf_feature_names[importance[:n_top]])


# TF-IDF 기준 상위 top 개수만큼 품사별 키워드 추출 (명사, 동사, 형용사)
noun,adj = topKeywords(okt,' '.join(tfidf_feature_names[importance[:n_top]]))

noun_top = 100
print(f'명사 top : {noun[:noun_top]}')
print()
adj_top = 10
print(f'형용사 top : {adj[:adj_top]}')

명사 top : ['리다', '색상', '색감', '구매', '느낌', '진짜', '보기', '핑크', '생각', '부분', '베이지', '컬러', '다른', '별로', '어깨', '블랙', '기모', '추천', '보이', '라인', '사진', '색도', '허리', '조금', '셔링', '여성', '소재', '디자인', '여리해', '팔뚝', '가슴', '사고', '재질', '버전', '색깔', '마음', '정말', '소매', '그냥', '살짝', '데이트', '쿨톤', '마감', '고민', '정도', '발렛베', '아이보리', '완전', '퍼빗', '가을', '배송', '체형', '약간', '겨울', '복숭아', '워머', '퀄리티', '실밥', '봄웜', '아주', '편이', '주문', '얼굴', '자체', '하나', '라이트', '부각', '속옷', '가격', '뮤트', '몸매', '길이', '리뷰', '부담', '발렛', '브라운', '웜톤', '숄더', '살구', '살이', '스타일', '오프', '다만', '상체', '마르다', '상품', '후기', '처음', '티셔츠', '걱정', '사이즈', '찰떡', '화면', '주름', '만족', '지그재그', '로잔', '오프', '숄더', '핑크색']

형용사 top : ['예쁘다', '좋다', '같다', '있다', '이쁘다', '아니다', '얇다', '괜찮다', '만족하다', '많다']


2-1. 형용사 사용

In [64]:
빈도분석(df, 'adj')

가장 많이 언급된 품사 (빈도순):
예쁘다: 270
같다: 243
좋다: 233
있다: 221
여리다: 184
이다: 142
이쁘다: 132
아니다: 56
없다: 47
얇다: 47


In [65]:
PMI(df, 'adj')

PMI 상위 품사:
알맞다 (문서 3): 14.03705282791151
부끄럽다 (문서 22): 14.03705282791151
가녀리다 (문서 47): 14.03705282791151
멀쩡하다 (문서 49): 14.03705282791151
낮다 (문서 57): 14.03705282791151
가득하다 (문서 64): 14.03705282791151
흡사하다 (문서 76): 14.03705282791151
실패하다 (문서 77): 14.03705282791151
좁다 (문서 93): 14.03705282791151
신나다 (문서 95): 14.03705282791151


In [66]:
WordEmbedding(df, 'adj')

상위단어
['예쁘다', '같다', '좋다', '있다', '여리다']

상위 단어와 유사한 품사:
예쁘다: [('좋다', 0.8136513233184814), ('같다', 0.7951556444168091), ('있다', 0.7873016595840454)]
같다: [('있다', 0.8036925196647644), ('좋다', 0.7963465452194214), ('예쁘다', 0.7951557040214539)]
좋다: [('여리다', 0.813860297203064), ('예쁘다', 0.8136512637138367), ('있다', 0.8094833493232727)]
있다: [('좋다', 0.8094834089279175), ('같다', 0.8036925792694092), ('예쁘다', 0.7873017191886902)]
여리다: [('좋다', 0.8138604760169983), ('예쁘다', 0.7706547379493713), ('같다', 0.7565170526504517)]


2-2. 명사 사용

In [67]:
빈도분석(df, 'noun')

가장 많이 언급된 품사 (빈도순):
더: 242
옷: 143
핏: 135
것: 128
보기: 124
색: 122
색상: 110
구매: 97
색감: 91
느낌: 87


In [68]:
PMI(df, 'noun')

PMI 상위 품사:
엄지손 (문서 0): 15.273544016239141
마법 (문서 1): 15.273544016239141
스판끼 (문서 1): 15.273544016239141
정석 (문서 2): 15.273544016239141
있어욤 (문서 2): 15.273544016239141
가끔 (문서 2): 15.273544016239141
한적 (문서 3): 15.273544016239141
갠 (문서 4): 15.273544016239141
꼽았어 (문서 4): 15.273544016239141
당일 (문서 5): 15.273544016239141


In [69]:
WordEmbedding(df, 'noun')

상위단어
['더', '옷', '핏', '것', '보기']

상위 단어와 유사한 품사:
더: [('느낌', 0.9941322803497314), ('옷', 0.9937158823013306), ('좀', 0.9936140179634094)]
옷: [('좀', 0.993878185749054), ('더', 0.9937160611152649), ('부분', 0.9932937622070312)]
핏: [('좀', 0.993990421295166), ('색상', 0.9932789206504822), ('옷', 0.9931802153587341)]
것: [('옷', 0.9931552410125732), ('좀', 0.9929196238517761), ('색', 0.992648184299469)]
보기: [('베이지', 0.9848901629447937), ('핏', 0.9843122363090515), ('핑크', 0.9834980964660645)]


2-3. all (형용사&명사)

In [70]:
빈도분석(df, 'all')

가장 많이 언급된 품사 (빈도순):
예쁘다: 270
같다: 243
더: 242
좋다: 233
있다: 221
여리다: 184
옷: 143
이다: 142
핏: 135
이쁘다: 132


In [71]:
PMI(df, 'all')

PMI 상위 품사:
엄지손 (문서 0): 15.528407178766605
마법 (문서 1): 15.528407178766605
스판끼 (문서 1): 15.528407178766605
정석 (문서 2): 15.528407178766605
있어욤 (문서 2): 15.528407178766605
가끔 (문서 2): 15.528407178766605
한적 (문서 3): 15.528407178766605
알맞다 (문서 3): 15.528407178766605
갠 (문서 4): 15.528407178766605
꼽았어 (문서 4): 15.528407178766605


In [72]:
WordEmbedding(df, 'all')

상위단어
['예쁘다', '같다', '더', '좋다', '있다']

상위 단어와 유사한 품사:
예쁘다: [('것', 0.9990606904029846), ('옷', 0.9990214705467224), ('느낌', 0.9990105032920837)]
같다: [('부분', 0.9989197254180908), ('더', 0.9988963007926941), ('구매', 0.9988637566566467)]
더: [('옷', 0.9995096921920776), ('핏', 0.9994602799415588), ('것', 0.9993990659713745)]
좋다: [('더', 0.9988873600959778), ('옷', 0.9988797903060913), ('것', 0.9988343119621277)]
있다: [('더', 0.999269962310791), ('색상', 0.9991440176963806), ('옷', 0.9990860223770142)]


##### 3. clean_count

In [76]:
okt = Okt()
df = pd.DataFrame(None, columns=['noun','adj','all'])
df.loc[0] = [1,2,3]
for idx,text in enumerate(product130108350['clean_count']):
    n,a = topKeywords(okt,text)
    df.loc[idx] = [' '.join(n), ' '.join(a), ' '.join(n)+' '+' '+' '.join(a)]

tfidf = TfidfVectorizer()
res = tfidf.fit_transform(df['all']).toarray()

# TF-IDF 기준 상위 n_top 개수 키워드 추출
n_top = 300
importance = np.argsort(np.asarray(res.sum(axis=0)).ravel())[::-1]
tfidf_feature_names = np.array(tfidf.get_feature_names_out())
# print(tfidf_feature_names[importance[:n_top]])


# TF-IDF 기준 상위 top 개수만큼 품사별 키워드 추출 (명사, 동사, 형용사)
noun,adj = topKeywords(okt,' '.join(tfidf_feature_names[importance[:n_top]]))

noun_top = 100
print(f'명사 top : {noun[:noun_top]}')
print()
adj_top = 10
print(f'형용사 top : {adj[:adj_top]}')

명사 top : ['리다', '색상', '구매', '느낌', '색감', '진짜', '핑크', '부분', '생각', '베이지', '컬러', '다른', '별로', '어깨', '블랙', '보기', '기모', '보이', '라인', '허리', '색도', '조금', '소재', '여성', '셔링', '디자인', '팔뚝', '추천', '사진', '여리해', '재질', '가슴', '사고', '버전', '마음', '색깔', '그냥', '소매', '정말', '마감', '데이트', '살짝', '고민', '정도', '아이보리', '쿨톤', '완전', '가을', '퍼빗', '겨울', '약간', '워머', '체형', '배송', '아주', '퀄리티', '발렛베', '편이', '복숭아', '실밥', '주문', '봄웜', '얼굴', '부각', '하나', '속옷', '자체', '길이', '몸매', '가격', '라이트', '숄더', '발렛', '부담', '뮤트', '살이', '스타일', '브라운', '다만', '오프', '살구', '웜톤', '상품', '마르다', '상체', '티셔츠', '걱정', '처음', '후기', '사이즈', '찰떡', '리뷰', '주름', '오프', '숄더', '지그재그', '브라', '화면', '정리', '핑크색']

형용사 top : ['예쁘다', '좋다', '같다', '있다', '이쁘다', '아니다', '얇다', '괜찮다', '없다', '많다']


3-1. 형용사 사용

In [77]:
빈도분석(df, 'adj')

가장 많이 언급된 품사 (빈도순):
예쁘다: 775
같다: 670
좋다: 662
있다: 639
여리다: 523
이다: 379
이쁘다: 370
아니다: 164
없다: 139
얇다: 139


In [78]:
PMI(df, 'adj')

PMI 상위 품사:
멀다 (문서 174): 14.395908885082225
무덥다 (문서 186): 14.395908885082225
용하다 (문서 244): 14.395908885082225
알맞다 (문서 3): 13.990443776974061
부끄럽다 (문서 22): 13.990443776974061
가녀리다 (문서 47): 13.990443776974061
멀쩡하다 (문서 49): 13.990443776974061
낮다 (문서 57): 13.990443776974061
가득하다 (문서 64): 13.990443776974061
흡사하다 (문서 76): 13.990443776974061


In [79]:
WordEmbedding(df, 'adj')

상위단어
['예쁘다', '같다', '좋다', '있다', '여리다']

상위 단어와 유사한 품사:
예쁘다: [('깔끔하다', 0.9985467791557312), ('고급스럽다', 0.9985451102256775), ('높다', 0.99852454662323)]
같다: [('은은하다', 0.9988837242126465), ('그렇다', 0.9987420439720154), ('있다', 0.9986683130264282)]
좋다: [('이쁘다', 0.9988527894020081), ('은은하다', 0.9987040162086487), ('이상하다', 0.9986475706100464)]
있다: [('그렇다', 0.998798668384552), ('이다', 0.9987853169441223), ('이쁘다', 0.9987225532531738)]
여리다: [('좋다', 0.9982301592826843), ('차분하다', 0.997882604598999), ('비슷하다', 0.9978200793266296)]


3-2. 명사 사용

In [80]:
빈도분석(df, 'noun')

가장 많이 언급된 품사 (빈도순):
색상: 313
색: 293
핏: 290
옷: 283
구매: 283
보기: 264
색감: 253
느낌: 252
핑크: 220
진짜: 216


In [81]:
PMI(df, 'noun')

PMI 상위 품사:
색더 (문서 15): 15.523480726462621
펜더 (문서 22): 15.523480726462621
꺄 (문서 23): 15.523480726462621
지인 (문서 27): 15.523480726462621
퍼킷 (문서 33): 15.523480726462621
밥 (문서 36): 15.523480726462621
신체 (문서 42): 15.523480726462621
잘잘 (문서 66): 15.523480726462621
줌 (문서 66): 15.523480726462621
퍼펙트 (문서 71): 15.523480726462621


In [82]:
WordEmbedding(df, 'noun')

상위단어
['색상', '색', '핏', '옷', '구매']

상위 단어와 유사한 품사:
색상: [('티', 0.9995359778404236), ('용', 0.9994273781776428), ('자주', 0.999410092830658)]
색: [('요', 0.99950110912323), ('맘', 0.999376118183136), ('색상', 0.9993494749069214)]
핏: [('진짜', 0.999527633190155), ('제', 0.999494731426239), ('편', 0.9994758367538452)]
옷: [('몸', 0.9994102716445923), ('완전', 0.9993626475334167), ('도', 0.9993539452552795)]
구매: [('것', 0.9994460344314575), ('사진', 0.9994036555290222), ('요', 0.9994017481803894)]


3-3. all

In [83]:
빈도분석(df, 'all')

가장 많이 언급된 품사 (빈도순):
예쁘다: 775
같다: 670
좋다: 662
있다: 639
여리다: 523
이다: 379
이쁘다: 370
색상: 313
색: 293
핏: 290


In [84]:
PMI(df, 'all')

PMI 상위 품사:
색더 (문서 15): 15.803966928491555
펜더 (문서 22): 15.803966928491555
꺄 (문서 23): 15.803966928491555
지인 (문서 27): 15.803966928491555
퍼킷 (문서 33): 15.803966928491555
밥 (문서 36): 15.803966928491555
신체 (문서 42): 15.803966928491555
잘잘 (문서 66): 15.803966928491555
줌 (문서 66): 15.803966928491555
퍼펙트 (문서 71): 15.803966928491555


In [85]:
WordEmbedding(df, 'all')

상위단어
['예쁘다', '같다', '좋다', '있다', '여리다']

상위 단어와 유사한 품사:
예쁘다: [('같다', 0.9975155591964722), ('좋다', 0.9968034029006958), ('있다', 0.9957370162010193)]
같다: [('이다', 0.9988465309143066), ('좋다', 0.9983950257301331), ('있다', 0.9977676868438721)]
좋다: [('있다', 0.998894989490509), ('같다', 0.9983950257301331), ('이다', 0.9979372024536133)]
있다: [('좋다', 0.9988949298858643), ('같다', 0.9977676868438721), ('이다', 0.9976714849472046)]
여리다: [('이다', 0.9946117997169495), ('예쁘다', 0.9943315386772156), ('좋다', 0.9933524131774902)]


##### 4. clean_tfidf

In [86]:
okt = Okt()
df = pd.DataFrame(None, columns=['noun','adj','all'])
df.loc[0] = [1,2,3]
for idx,text in enumerate(product130108350['clean_tfidf']):
    n,a = topKeywords(okt,text)
    df.loc[idx] = [' '.join(n), ' '.join(a), ' '.join(n)+' '+' '+' '.join(a)]

tfidf = TfidfVectorizer()
res = tfidf.fit_transform(df['all']).toarray()

# TF-IDF 기준 상위 n_top 개수 키워드 추출
n_top = 300
importance = np.argsort(np.asarray(res.sum(axis=0)).ravel())[::-1]
tfidf_feature_names = np.array(tfidf.get_feature_names_out())
# print(tfidf_feature_names[importance[:n_top]])


# TF-IDF 기준 상위 top 개수만큼 품사별 키워드 추출 (명사, 동사, 형용사)
noun,adj = topKeywords(okt,' '.join(tfidf_feature_names[importance[:n_top]]))

noun_top = 100
print(f'명사 top : {noun[:noun_top]}')
print()
adj_top = 10
print(f'형용사 top : {adj[:adj_top]}')

명사 top : ['리다', '색상', '구매', '느낌', '색감', '진짜', '핑크', '부분', '생각', '베이지', '컬러', '다른', '별로', '어깨', '블랙', '보기', '기모', '보이', '라인', '허리', '색도', '조금', '소재', '여성', '셔링', '디자인', '팔뚝', '추천', '사진', '여리해', '재질', '가슴', '사고', '버전', '마음', '색깔', '그냥', '소매', '정말', '마감', '데이트', '살짝', '고민', '정도', '아이보리', '쿨톤', '완전', '가을', '퍼빗', '겨울', '약간', '워머', '체형', '배송', '아주', '퀄리티', '발렛베', '편이', '복숭아', '실밥', '주문', '봄웜', '얼굴', '부각', '하나', '속옷', '자체', '길이', '몸매', '가격', '라이트', '숄더', '발렛', '부담', '뮤트', '살이', '스타일', '브라운', '다만', '오프', '살구', '웜톤', '상품', '마르다', '상체', '티셔츠', '걱정', '처음', '후기', '사이즈', '찰떡', '리뷰', '주름', '오프', '숄더', '지그재그', '브라', '화면', '정리', '핑크색']

형용사 top : ['예쁘다', '좋다', '같다', '있다', '이쁘다', '아니다', '얇다', '괜찮다', '없다', '많다']


4-1. 형용사

In [87]:
빈도분석(df, 'adj')

가장 많이 언급된 품사 (빈도순):
예쁘다: 775
같다: 670
좋다: 662
있다: 639
여리다: 523
이다: 379
이쁘다: 370
아니다: 164
없다: 139
얇다: 139


In [88]:
PMI(df, 'adj')

PMI 상위 품사:
멀다 (문서 174): 14.395908885082225
무덥다 (문서 186): 14.395908885082225
용하다 (문서 244): 14.395908885082225
알맞다 (문서 3): 13.990443776974061
부끄럽다 (문서 22): 13.990443776974061
가녀리다 (문서 47): 13.990443776974061
멀쩡하다 (문서 49): 13.990443776974061
낮다 (문서 57): 13.990443776974061
가득하다 (문서 64): 13.990443776974061
흡사하다 (문서 76): 13.990443776974061


In [89]:
WordEmbedding(df, 'adj')

상위단어
['예쁘다', '같다', '좋다', '있다', '여리다']

상위 단어와 유사한 품사:
예쁘다: [('깔끔하다', 0.9985467791557312), ('고급스럽다', 0.9985451102256775), ('높다', 0.99852454662323)]
같다: [('은은하다', 0.9988837242126465), ('그렇다', 0.9987420439720154), ('있다', 0.9986683130264282)]
좋다: [('이쁘다', 0.9988527894020081), ('은은하다', 0.9987040162086487), ('이상하다', 0.9986475706100464)]
있다: [('그렇다', 0.998798668384552), ('이다', 0.9987853169441223), ('이쁘다', 0.9987225532531738)]
여리다: [('좋다', 0.9982301592826843), ('차분하다', 0.997882604598999), ('비슷하다', 0.9978200793266296)]


4-2. 명사

In [90]:
빈도분석(df, 'noun')

가장 많이 언급된 품사 (빈도순):
색상: 313
색: 293
핏: 290
옷: 283
구매: 283
보기: 264
색감: 253
느낌: 252
핑크: 220
진짜: 216


In [91]:
PMI(df, 'noun')

PMI 상위 품사:
색더 (문서 15): 15.523480726462621
펜더 (문서 22): 15.523480726462621
꺄 (문서 23): 15.523480726462621
지인 (문서 27): 15.523480726462621
퍼킷 (문서 33): 15.523480726462621
밥 (문서 36): 15.523480726462621
신체 (문서 42): 15.523480726462621
잘잘 (문서 66): 15.523480726462621
줌 (문서 66): 15.523480726462621
퍼펙트 (문서 71): 15.523480726462621


In [92]:
WordEmbedding(df, 'noun')

상위단어
['색상', '색', '핏', '옷', '구매']

상위 단어와 유사한 품사:
색상: [('맘', 0.9994573593139648), ('티', 0.9994523525238037), ('느낌', 0.999371349811554)]
색: [('요', 0.9995260238647461), ('맘', 0.9994467496871948), ('부', 0.9993268251419067)]
핏: [('허리', 0.9994962811470032), ('편', 0.9994896054267883), ('주름', 0.9994733929634094)]
옷: [('몸', 0.9994579553604126), ('완전', 0.9994567036628723), ('정도', 0.9994160532951355)]
구매: [('것', 0.9992544054985046), ('고민', 0.9992441534996033), ('컬러', 0.9992119073867798)]


4-3. all

In [93]:
빈도분석(df, 'all')

가장 많이 언급된 품사 (빈도순):
예쁘다: 775
같다: 670
좋다: 662
있다: 639
여리다: 523
이다: 379
이쁘다: 370
색상: 313
색: 293
핏: 290


In [94]:
PMI(df, 'all')

PMI 상위 품사:
색더 (문서 15): 15.803966928491555
펜더 (문서 22): 15.803966928491555
꺄 (문서 23): 15.803966928491555
지인 (문서 27): 15.803966928491555
퍼킷 (문서 33): 15.803966928491555
밥 (문서 36): 15.803966928491555
신체 (문서 42): 15.803966928491555
잘잘 (문서 66): 15.803966928491555
줌 (문서 66): 15.803966928491555
퍼펙트 (문서 71): 15.803966928491555


In [95]:
WordEmbedding(df, 'all')

상위단어
['예쁘다', '같다', '좋다', '있다', '여리다']

상위 단어와 유사한 품사:
예쁘다: [('좋다', 0.9969079494476318), ('같다', 0.9960328936576843), ('여리다', 0.9954383969306946)]
같다: [('좋다', 0.9987727403640747), ('있다', 0.9982931613922119), ('이다', 0.9977961182594299)]
좋다: [('같다', 0.9987727999687195), ('있다', 0.9976831078529358), ('예쁘다', 0.9969078898429871)]
있다: [('같다', 0.9982931613922119), ('이다', 0.9978074431419373), ('좋다', 0.9976831078529358)]
여리다: [('좋다', 0.995499312877655), ('예쁘다', 0.9954385161399841), ('같다', 0.9938734173774719)]


#### 상품 : product147921597

In [98]:
product147921597 = review[review.상품ID == 147921597]
product147921597 = product147921597[['상품명', '리뷰 코멘트', 'clean_count', 'clean_tfidf']]
product147921597.head(1)

,상품명,리뷰 코멘트,clean_count,clean_tfidf
500,리얼코코 [단독] [봄신상🌸/브이넥&amp;라운드넥/made] 시오 니트 - 20 ...,"여리여리한 브이넥 니트, 겨울부터 초봄까지 완벽한 아이템! 🌿 이 브이넥 니트는 ...",강조해 강조해 줍니다 겨울부터 겨울부터 초봄까지 깔끔하면서도 깔끔하면서도 세련된 느...,강조해 강조해 줍니다 겨울부터 겨울부터 초봄까지 깔끔하면서도 깔끔하면서도 세련된 느...


In [101]:
product147921597['상품명'][500]

'리얼코코 [단독] [봄신상🌸/브이넥&amp;라운드넥/made] 시오 니트 - 20 Color (하객룩/레이어드/출근룩/롱슬리브/봄옷/오피스룩/상의)'

##### 1. 상품명

In [102]:
okt = Okt()
df = pd.DataFrame(None, columns=['noun','adj','all'])
df.loc[0] = [1,2,3]
for idx,text in enumerate(product147921597['상품명']):
    n,a = topKeywords(okt,text)
    df.loc[idx] = [' '.join(n), ' '.join(a), ' '.join(n)+' '+' '+' '.join(a)]

tfidf = TfidfVectorizer()
res = tfidf.fit_transform(df['all']).toarray()

# TF-IDF 기준 상위 n_top 개수 키워드 추출
n_top = 300
importance = np.argsort(np.asarray(res.sum(axis=0)).ravel())[::-1]
tfidf_feature_names = np.array(tfidf.get_feature_names_out())
# print(tfidf_feature_names[importance[:n_top]])


# TF-IDF 기준 상위 top 개수만큼 품사별 키워드 추출 (명사, 동사, 형용사)
noun,adj = topKeywords(okt,' '.join(tfidf_feature_names[importance[:n_top]]))

noun_top = 100
print(f'명사 top : {noun[:noun_top]}')
print()
adj_top = 10
print(f'형용사 top : {adj[:adj_top]}')

명사 top : ['하객', '코코', '출근', '오피스', '신상', '시오', '슬리브', '상의', '브이넥', '봄옷', '리얼', '레이어', '라운드', '단독', '니트']

형용사 top : []


##### 2. clean_count

In [103]:
okt = Okt()
df = pd.DataFrame(None, columns=['noun','adj','all'])
df.loc[0] = [1,2,3]
for idx,text in enumerate(product147921597['clean_count']):
    n,a = topKeywords(okt,text)
    df.loc[idx] = [' '.join(n), ' '.join(a), ' '.join(n)+' '+' '+' '.join(a)]

tfidf = TfidfVectorizer()
res = tfidf.fit_transform(df['all']).toarray()

# TF-IDF 기준 상위 n_top 개수 키워드 추출
n_top = 300
importance = np.argsort(np.asarray(res.sum(axis=0)).ravel())[::-1]
tfidf_feature_names = np.array(tfidf.get_feature_names_out())
# print(tfidf_feature_names[importance[:n_top]])


# TF-IDF 기준 상위 top 개수만큼 품사별 키워드 추출 (명사, 동사, 형용사)
noun,adj = topKeywords(okt,' '.join(tfidf_feature_names[importance[:n_top]]))

noun_top = 100
print(f'명사 top : {noun[:noun_top]}')
print()
adj_top = 10
print(f'형용사 top : {adj[:adj_top]}')

명사 top : ['니트', '구매', '색상', '브이넥', '보기', '느낌', '아이보리', '생각', '색감', '진짜', '재질', '사진', '고민', '길이', '보풀', '다른', '정말', '소재', '퀄리티', '마음', '하나', '정도', '컬러', '가격', '두께', '겨울', '체형', '허리', '얼굴', '조금', '기장', '어깨', '라운드', '살짝', '리뷰', '후기', '완전', '단독', '걱정', '보리', '스카이', '배송', '그레이', '약간', '가슴', '추천', '멜론', '리다', '포인트', '디자인', '사이즈', '만족', '색도', '부분', '색깔', '블루', '하늘색', '주문', '편이', '화면', '이유', '제품', '자주', '밑단', '피부', '그냥', '선택', '핑크', '레드', '일단', '바지', '당첨', '촉감', '사고', '처음', '착용', '보이', '다만', '무난', '거리', '별로', '파스텔', '일리', '쿨톤', '라인', '추가', '셔츠', '버전', '짜임', '가성', '아주', '정전기', '대비', '카시', '자체', '레이어', '여리해', '화이트', '크롭', '역시']

형용사 top : ['좋다', '같다', '있다', '예쁘다', '부드럽다', '이쁘다', '적당하다', '도톰하다', '없다', '따뜻하다']


2-1. 형용사 사용

In [104]:
빈도분석(df, 'adj')

가장 많이 언급된 품사 (빈도순):
좋다: 470
같다: 422
있다: 340
예쁘다: 291
이다: 207
부드럽다: 156
도톰하다: 124
이쁘다: 124
적당하다: 119
없다: 104


In [105]:
WordEmbedding(df, 'adj')

상위단어
['좋다', '같다', '있다', '예쁘다', '이다']

상위 단어와 유사한 품사:
좋다: [('없다', 0.9994710087776184), ('있다', 0.9993377327919006), ('같다', 0.9993017315864563)]
같다: [('없다', 0.9993231892585754), ('있다', 0.9993049502372742), ('좋다', 0.9993016123771667)]
있다: [('이다', 0.9993889927864075), ('없다', 0.999367892742157), ('좋다', 0.9993376135826111)]
예쁘다: [('있다', 0.9993230700492859), ('같다', 0.9992616176605225), ('좋다', 0.9992237687110901)]
이다: [('있다', 0.9993889331817627), ('같다', 0.9992595911026001), ('좋다', 0.9992237687110901)]


2-2. 명사 사용

In [106]:
빈도분석(df, 'noun')

가장 많이 언급된 품사 (빈도순):
니트: 372
보기: 310
구매: 265
색상: 258
더: 221
브이넥: 182
색: 180
핏: 158
느낌: 150
아이보리: 141


In [107]:
WordEmbedding(df, 'noun')

상위단어
['니트', '보기', '구매', '색상', '더']

상위 단어와 유사한 품사:
니트: [('보기', 0.9997350573539734), ('색', 0.9996762871742249), ('정도', 0.9996649622917175)]
보기: [('니트', 0.9997351169586182), ('색감', 0.9996520280838013), ('진짜', 0.9996509552001953)]
구매: [('색감', 0.9996543526649475), ('니트', 0.9996497631072998), ('색', 0.9996470808982849)]
색상: [('보기', 0.9996347427368164), ('브이넥', 0.9996222853660583), ('색', 0.9996176958084106)]
더: [('두께', 0.9996459484100342), ('니트', 0.9996386766433716), ('보기', 0.9996333122253418)]


2-3. all

In [108]:
빈도분석(df, 'all')

가장 많이 언급된 품사 (빈도순):
좋다: 470
같다: 422
니트: 372
있다: 340
보기: 310
예쁘다: 291
구매: 265
색상: 258
더: 221
이다: 207


In [109]:
WordEmbedding(df, 'all')

상위단어
['좋다', '같다', '니트', '있다', '보기']

상위 단어와 유사한 품사:
좋다: [('있다', 0.9981806874275208), ('예쁘다', 0.9963506460189819), ('이쁘다', 0.9944601058959961)]
같다: [('부드럽다', 0.9991528391838074), ('따뜻하다', 0.9988184571266174), ('이다', 0.9987478256225586)]
니트: [('보기', 0.9996867179870605), ('더', 0.9996697306632996), ('구매', 0.9995707869529724)]
있다: [('좋다', 0.9981807470321655), ('예쁘다', 0.9971825480461121), ('이쁘다', 0.9960887432098389)]
보기: [('두께', 0.9996975660324097), ('니트', 0.9996866583824158), ('정말', 0.9996705651283264)]


##### 3. clean_tfidf

In [110]:
okt = Okt()
df = pd.DataFrame(None, columns=['noun','adj','all'])
df.loc[0] = [1,2,3]
for idx,text in enumerate(product147921597['clean_tfidf']):
    n,a = topKeywords(okt,text)
    df.loc[idx] = [' '.join(n), ' '.join(a), ' '.join(n)+' '+' '+' '.join(a)]

tfidf = TfidfVectorizer()
res = tfidf.fit_transform(df['all']).toarray()

# TF-IDF 기준 상위 n_top 개수 키워드 추출
n_top = 300
importance = np.argsort(np.asarray(res.sum(axis=0)).ravel())[::-1]
tfidf_feature_names = np.array(tfidf.get_feature_names_out())
# print(tfidf_feature_names[importance[:n_top]])


# TF-IDF 기준 상위 top 개수만큼 품사별 키워드 추출 (명사, 동사, 형용사)
noun,adj = topKeywords(okt,' '.join(tfidf_feature_names[importance[:n_top]]))

noun_top = 100
print(f'명사 top : {noun[:noun_top]}')
print()
adj_top = 10
print(f'형용사 top : {adj[:adj_top]}')

명사 top : ['니트', '구매', '색상', '브이넥', '보기', '느낌', '아이보리', '생각', '색감', '진짜', '재질', '사진', '고민', '길이', '보풀', '다른', '정말', '소재', '퀄리티', '마음', '하나', '정도', '컬러', '가격', '두께', '겨울', '체형', '허리', '얼굴', '조금', '기장', '어깨', '라운드', '살짝', '리뷰', '후기', '완전', '단독', '걱정', '보리', '스카이', '배송', '그레이', '약간', '가슴', '추천', '멜론', '리다', '포인트', '디자인', '사이즈', '만족', '색도', '부분', '색깔', '블루', '하늘색', '주문', '편이', '화면', '이유', '제품', '자주', '밑단', '피부', '그냥', '선택', '핑크', '레드', '일단', '바지', '당첨', '촉감', '사고', '처음', '착용', '보이', '다만', '무난', '거리', '별로', '파스텔', '일리', '쿨톤', '라인', '추가', '셔츠', '버전', '짜임', '가성', '아주', '정전기', '대비', '카시', '자체', '레이어', '여리해', '화이트', '크롭', '역시']

형용사 top : ['좋다', '같다', '있다', '예쁘다', '부드럽다', '이쁘다', '적당하다', '도톰하다', '없다', '따뜻하다']


3-1. 형용사 사용

In [111]:
빈도분석(df, 'adj')

가장 많이 언급된 품사 (빈도순):
좋다: 470
같다: 422
있다: 340
예쁘다: 291
이다: 207
부드럽다: 156
도톰하다: 124
이쁘다: 124
적당하다: 119
없다: 104


In [112]:
WordEmbedding(df, 'adj')

상위단어
['좋다', '같다', '있다', '예쁘다', '이다']

상위 단어와 유사한 품사:
좋다: [('없다', 0.9994710087776184), ('있다', 0.9993377327919006), ('같다', 0.9993017315864563)]
같다: [('없다', 0.9993231892585754), ('있다', 0.9993049502372742), ('좋다', 0.9993016123771667)]
있다: [('이다', 0.9993889927864075), ('없다', 0.999367892742157), ('좋다', 0.9993376135826111)]
예쁘다: [('있다', 0.9993230700492859), ('같다', 0.9992616176605225), ('좋다', 0.9992237687110901)]
이다: [('있다', 0.9993889331817627), ('같다', 0.9992595911026001), ('좋다', 0.9992237687110901)]


3-2. 명사 사용

In [113]:
빈도분석(df, 'noun')

가장 많이 언급된 품사 (빈도순):
니트: 372
보기: 310
구매: 265
색상: 258
더: 221
브이넥: 182
색: 180
핏: 158
느낌: 150
아이보리: 141


In [114]:
WordEmbedding(df, 'noun')

상위단어
['니트', '보기', '구매', '색상', '더']

상위 단어와 유사한 품사:
니트: [('보기', 0.999737024307251), ('색', 0.9996783137321472), ('정도', 0.9996628165245056)]
보기: [('니트', 0.9997371435165405), ('색감', 0.9996544122695923), ('브이넥', 0.9996515512466431)]
구매: [('색감', 0.9996523857116699), ('색', 0.9996466040611267), ('니트', 0.9996461868286133)]
색상: [('보기', 0.9996377229690552), ('브이넥', 0.9996278882026672), ('색', 0.9996205568313599)]
더: [('두께', 0.9996452331542969), ('니트', 0.9996398687362671), ('보기', 0.999635636806488)]


3-3. all

In [115]:
빈도분석(df, 'all')

가장 많이 언급된 품사 (빈도순):
좋다: 470
같다: 422
니트: 372
있다: 340
보기: 310
예쁘다: 291
구매: 265
색상: 258
더: 221
이다: 207


In [116]:
WordEmbedding(df, 'all')

상위단어
['좋다', '같다', '니트', '있다', '보기']

상위 단어와 유사한 품사:
좋다: [('있다', 0.9982182383537292), ('예쁘다', 0.9963417649269104), ('이쁘다', 0.9944908618927002)]
같다: [('부드럽다', 0.9991495609283447), ('따뜻하다', 0.9988108277320862), ('이다', 0.9987550973892212)]
니트: [('보기', 0.9996870160102844), ('더', 0.9996736645698547), ('구매', 0.9995772838592529)]
있다: [('좋다', 0.9982181787490845), ('예쁘다', 0.9971246719360352), ('이쁘다', 0.9960204362869263)]
보기: [('두께', 0.9997004866600037), ('니트', 0.9996871948242188), ('정말', 0.9996541142463684)]


## 결론

In [1]:
review_keyword = review[['상품명', 'clean_count', '상품ID']]
review_keyword

NameError: name 'review' is not defined

In [ ]:
review_keyword = review_keyword[review_keyword['clean_count'].notna()]   # 결측이 아닌 것만 사용해야 됨..

In [133]:
import pandas as pd
import numpy as np
from collections import Counter
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

def topKeywords(okt, text):
    nouns = []
    adjs = []
    for word, pos in okt.pos(text):
        if pos == 'Noun':
            nouns.append(word)
        elif pos == 'Adjective':
            adjs.append(word)
    return nouns, adjs

def extract_keywords(product_df):
    okt = Okt()
    result_list = []
    
    for product_id, group in product_df.groupby('상품ID'):
        # 상품명 키워드 추출
        df_name = pd.DataFrame(columns=['all'])
        df_name['all'] = group['상품명'].apply(lambda x: ' '.join(topKeywords(okt, x)[0]))
        tfidf = TfidfVectorizer()
        res = tfidf.fit_transform(df_name['all']).toarray()
        importance = np.argsort(np.asarray(res.sum(axis=0)).ravel())[::-1]
        tfidf_feature_names = np.array(tfidf.get_feature_names_out())
        name_keywords = tfidf_feature_names[importance[:100]]
        
        # 리뷰 키워드 추출
        df_review = pd.DataFrame(columns=['all'])
        df_review['all'] = group['clean_count'].apply(lambda x: ' '.join(topKeywords(okt, x)[0] + topKeywords(okt, x)[1]))
        tfidf = TfidfVectorizer()
        res = tfidf.fit_transform(df_review['all']).toarray()
        importance = np.argsort(np.asarray(res.sum(axis=0)).ravel())[::-1]
        tfidf_feature_names = np.array(tfidf.get_feature_names_out())
        review_noun, review_adj = topKeywords(okt, ' '.join(tfidf_feature_names[importance[:300]]))
        
        # Word2Vec을 활용한 형용사 유사 단어
        df_adj = pd.DataFrame({'adj': group['clean_count'].apply(lambda x: ' '.join(topKeywords(okt, x)[1]))})
        sentences = df_adj['adj'].str.split().tolist()
        model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
        word_counts = Counter(df_adj['adj'].str.split().explode())
        top_words = [word for word, count in word_counts.most_common(5)]
        similar_words = {word: model.wv.most_similar(word, topn=3) for word in top_words}
        
        # 결과 리스트에 추가
        result_list.append({
            '상품ID': product_id,
            '상품명 키워드': ', '.join(name_keywords),
            '리뷰 키워드 (명사)': ', '.join(review_noun[:100]),
            '리뷰 키워드 (형용사)': ', '.join(review_adj[:10]),
            '형용사 유사 단어': str(similar_words)
        })
    
    # 리스트를 데이터프레임으로 변환
    result_df = pd.DataFrame(result_list)
    return result_df


In [134]:
a = extract_keywords(review_keyword)

In [135]:
a.to_csv("키워드추출.csv", index=False, encoding="utf-8-sig")

In [136]:
a

,상품ID,상품명 키워드,리뷰 키워드 (명사),리뷰 키워드 (형용사),형용사 유사 단어
0,100457185.0,"팬츠, 패럴, 트렌디, 부츠, 밴딩, 기획, 공항","길이, 바지, 재질, 기장, 구매, 생각, 다리, 진짜, 느낌, 밴딩, 사이즈, 허...","같아요, 좋아요, 편하고, 편해요, 편하게, 입니다, 좋고, 예뻐요, 길어, 날씬해","{'같아요': [('좋아요', 0.9903361797332764), ('입니다', ..."
1,100468216.0,"페이보릿, 클래식, 진행, 상품, 베스트, 배송, 무료, 메리, 리뷰, 라운드, 단독","자켓, 사이즈, 버핏, 구매, 어깨, 퀄리티, 가격, 느낌, 생각, 차콜, 마음, ...","같아요, 좋아요, 입니다, 있어서, 좋은, 좋고, 적당히, 예뻐요, 좋아서, 만족합니다","{'같아요': [('좋아요', 0.9966840147972107), ('입니다', ..."
2,100480591.0,"호피, 폭주, 주얼, 주문, 일리, 여성, 에코, 숄더백, 빈티, 발송, 매듭, 리...","가방, 생각, 포인트, 호피, 가격, 크로스, 배송, 물건, 구매, 디자인, 바닥,...","좋아요, 같아요, 있어서, 이뻐요, 예뻐요, 편하게, 좋고, 좋은, 가볍고, 있고","{'좋아요': [('같아요', 0.8993897438049316), ('있어서', ..."
3,100781760.0,"헬스, 필라테스, 토드, 출고, 추가, 요가, 여행, 빅백, 보스턴, 보라, 당일, 가방","여행, 공간, 가방, 수납, 사용, 신발, 방수, 완전, 주머니, 색깔, 진짜, 마...","좋아요, 있어서, 같아요, 유용하게, 좋고, 좋았어요, 입니다, 넓고, 가볍고, 좋은","{'좋아요': [('많은거', 0.29679858684539795), ('빨라요',..."
4,101062215.0,"기본, 하객, 팬츠, 겨울, 구매, 기장, 다리, 라이프, 미친핏, 밍크, 베니, ...","바지, 허리, 사이즈, 길이, 구매, 버전, 재질, 다리, 진짜, 겨울, 슬랙스, ...","같아요, 좋아요, 입니다, 편하고, 있어서, 좋아서, 길어, 있는, 편해요, 날씬해","{'같아요': [('좋아요', 0.9990600943565369), ('좋아서', ..."
...,...,...,...,...,...
284,154238846.0,"카디건, 스파오, 라운드","사이즈, 가디건, 구매, 재질, 생각, 느낌, 가격, 보풀, 색상, 보기, 다른, ...","같아요, 좋아요, 좋고, 입니다, 부드럽고, 좋은, 좋아서, 있는, 예뻐요, 있어서","{'같아요': [('좋아요', 0.9991394281387329), ('있는', 0..."
285,156434505.0,"후아유, 헨리, 티셔츠, 골지","사이즈, 구매, 생각, 여름, 네이비, 재질, 다른, 살짝, 정도, 퀄리티, 색상,...","같아요, 예뻐요, 좋아요, 있는, 좋고, 이뻐요, 작게, 예쁘고, 있어서, 적당히","{'같아요': [('있으신', 0.489641398191452), ('적당해서', ..."
286,158195942.0,"틴팅, 태그, 증정, 이상, 블러, 뷰튜버, 밀크, 구매","가격, 느낌, 색상, 구매, 제품, 베이스, 발색, 발림, 파우치, 생각, 추천, ...","좋아요, 같아요, 좋은, 부드럽게, 예쁘고, 없어서, 예뻐요, 부드러워서, 딱이에요...","{'좋아요': [('촉촉하게', 0.2943156957626343), ('괜찮고',..."
287,158197795.0,"팔레트, 태그, 아이, 글림","화면, 색감, 별로, 구애, 여뮽, 뮤트, 베이스, 글리터, 사용, 주문, 포인트,...","같아요, 꼼꼼하게, 빠르고, 좋아요, 가까워서, 괜찮을거, 입니당, 이뻐서, 작았는...","{'같아요': [('작았는데', 0.21613812446594238), ('이뿔듯'..."


In [6]:
keyword = pd.read_csv("키워드추출.csv")
keyword

,상품ID,상품명 키워드,리뷰 키워드 (명사),리뷰 키워드 (형용사),형용사 유사 단어,스타일
0,100457185,"팬츠, 패럴, 트렌디, 부츠, 밴딩, 기획, 공항","길이, 바지, 재질, 기장, 구매, 생각, 다리, 진짜, 느낌, 밴딩, 사이즈, 허...","같아요, 좋아요, 편하고, 편해요, 편하게, 입니다, 좋고, 예뻐요, 길어, 날씬해","{'같아요': [('좋아요', 0.9903361797332764), ('입니다', ...",캐주얼
1,100468216,"페이보릿, 클래식, 진행, 상품, 베스트, 배송, 무료, 메리, 리뷰, 라운드, 단독","자켓, 사이즈, 버핏, 구매, 어깨, 퀄리티, 가격, 느낌, 생각, 차콜, 마음, ...","같아요, 좋아요, 입니다, 있어서, 좋은, 좋고, 적당히, 예뻐요, 좋아서, 만족합니다","{'같아요': [('좋아요', 0.9966840147972107), ('입니다', ...",미니멀
2,100480591,"호피, 폭주, 주얼, 주문, 일리, 여성, 에코, 숄더백, 빈티, 발송, 매듭, 리...","가방, 생각, 포인트, 호피, 가격, 크로스, 배송, 물건, 구매, 디자인, 바닥,...","좋아요, 같아요, 있어서, 이뻐요, 예뻐요, 편하게, 좋고, 좋은, 가볍고, 있고","{'좋아요': [('같아요', 0.8993897438049316), ('있어서', ...",빈티지
3,100781760,"헬스, 필라테스, 토드, 출고, 추가, 요가, 여행, 빅백, 보스턴, 보라, 당일, 가방","여행, 공간, 가방, 수납, 사용, 신발, 방수, 완전, 주머니, 색깔, 진짜, 마...","좋아요, 있어서, 같아요, 유용하게, 좋고, 좋았어요, 입니다, 넓고, 가볍고, 좋은","{'좋아요': [('많은거', 0.29679858684539795), ('빨라요',...",캐주얼
4,101062215,"기본, 하객, 팬츠, 겨울, 구매, 기장, 다리, 라이프, 미친핏, 밍크, 베니, ...","바지, 허리, 사이즈, 길이, 구매, 버전, 재질, 다리, 진짜, 겨울, 슬랙스, ...","같아요, 좋아요, 입니다, 편하고, 있어서, 좋아서, 길어, 있는, 편해요, 날씬해","{'같아요': [('좋아요', 0.9990600943565369), ('좋아서', ...",미니멀
...,...,...,...,...,...,...
284,154238846,"카디건, 스파오, 라운드","사이즈, 가디건, 구매, 재질, 생각, 느낌, 가격, 보풀, 색상, 보기, 다른, ...","같아요, 좋아요, 좋고, 입니다, 부드럽고, 좋은, 좋아서, 있는, 예뻐요, 있어서","{'같아요': [('좋아요', 0.9991394281387329), ('있는', 0...",NaN
285,156434505,"후아유, 헨리, 티셔츠, 골지","사이즈, 구매, 생각, 여름, 네이비, 재질, 다른, 살짝, 정도, 퀄리티, 색상,...","같아요, 예뻐요, 좋아요, 있는, 좋고, 이뻐요, 작게, 예쁘고, 있어서, 적당히","{'같아요': [('있으신', 0.489641398191452), ('적당해서', ...",NaN
286,158195942,"틴팅, 태그, 증정, 이상, 블러, 뷰튜버, 밀크, 구매","가격, 느낌, 색상, 구매, 제품, 베이스, 발색, 발림, 파우치, 생각, 추천, ...","좋아요, 같아요, 좋은, 부드럽게, 예쁘고, 없어서, 예뻐요, 부드러워서, 딱이에요...","{'좋아요': [('촉촉하게', 0.2943156957626343), ('괜찮고',...",NaN
287,158197795,"팔레트, 태그, 아이, 글림","화면, 색감, 별로, 구애, 여뮽, 뮤트, 베이스, 글리터, 사용, 주문, 포인트,...","같아요, 꼼꼼하게, 빠르고, 좋아요, 가까워서, 괜찮을거, 입니당, 이뻐서, 작았는...","{'같아요': [('작았는데', 0.21613812446594238), ('이뿔듯'...",NaN


In [8]:

# 한 파일당 저장할 행 수
chunk_size = 20

# 데이터프레임을 나누어 CSV 저장
for idx, start in enumerate(range(0, len(keyword), chunk_size), start=1):
    keyword.iloc[start:start+chunk_size].to_csv(f'data_part_{idx}.csv', index=False, encoding='utf-8-sig')

In [10]:
forders = ["data_part_1.csv",
    "data_part_2.csv",
    "data_part_3.csv",
    "data_part_4.csv",
    "data_part_5.csv",
    "data_part_6.csv",
    "data_part_7.csv",
    "data_part_8.csv",
    "data_part_9.csv",
    "data_part_10.csv",
    "data_part_11.csv",
    "data_part_12.csv",
    "data_part_13.csv",
    "data_part_14.csv",
    "data_part_15.csv"]

키워드_분류 = pd.DataFrame()
for files in forders:
    df = pd.read_csv(files, encoding="utf-8-sig")
    키워드_분류 = pd.concat([키워드_분류, df])
    
키워드_분류 = 키워드_분류.reset_index(drop=True)
키워드_분류 = 키워드_분류[['상품ID', '스타일']]

In [11]:
키워드_분류

,상품ID,스타일
0,100457185,캐주얼
1,100468216,미니멀
2,100480591,빈티지
3,100781760,캐주얼
4,101062215,미니멀
...,...,...
284,154238846,미니멀
285,156434505,미니멀
286,158195942,러블리
287,158197795,미니멀


In [12]:
product.head()

,순위,카테고리,세부 카테고리,브랜드,상품명,정가,할인율,할인가,최저가,평균 리뷰 점수,리뷰 개수,배송,링크,상품ID
0,1,의류,상의,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,43200,0.31,37300.0,29840,4.8,4509,일반배송무료배송,https://zigzag.kr/catalog/products/130108350,130108350
1,2,의류,상의,리얼코코,리얼코코 [단독] [봄신상🌸/브이넥&amp;라운드넥/made] 시오 니트 - 20 ...,51000,0.57,24630.0,22160,4.8,16998,직진배송,https://zigzag.kr/catalog/products/147921597,147921597
2,3,의류,하의,고고싱,고고싱 [역대급53만장🏆11컬러/부츠컷슬랙스🌷봄팬츠] 반하이 모찌슬랙스 #세미부츠컷...,34900,0.36,24800.0,22320,4.7,77846,직진배송,https://zigzag.kr/catalog/products/113041967,113041967
3,4,의류,상의,데일리쥬,데일리쥬 [단독][1만장판매/MADE] 라스카 레이어드 텐셀 라운드 티셔츠 (워머옵...,30000,0.36,21370.0,19230,4.7,5248,직진배송,https://zigzag.kr/catalog/products/128943162,128943162
4,5,의류,상의,마이온샵,마이온샵 [BEST][누적2만장돌파🥇][4타입] #MADE 메로트 골지 니트티 (4...,43000,0.42,31160.0,24920,4.7,3215,직진배송,https://zigzag.kr/catalog/products/129147932,129147932


In [16]:
product_분류포함 = pd.merge(product, 키워드_분류, on='상품ID', how='outer')
product_분류포함.to_csv("product_분류포함.csv", index=False, encoding='utf-8-sig')

In [3]:
product_분류포함 = pd.read_csv("product_분류포함.csv")

In [4]:
product_분류포함.columns

Index(['순위', '카테고리', '세부 카테고리', '브랜드', '상품명', '정가', '할인율', '할인가', '최저가',
       '평균 리뷰 점수', '리뷰 개수', '배송', '링크', '상품ID', '스타일'],
      dtype='object')

In [10]:
product_분류포함.query("카테고리 == '의류'").groupby(['스타일', '세부 카테고리']).size()

스타일  세부 카테고리
러블리  상의         29
     치마          8
미니멀  상의         30
     아우터         3
     치마          1
     하의          8
빈티지  상의          4
     아우터         9
     치마          2
     하의         15
스트릿  하의          2
캐주얼  상의          7
     아우터         1
     치마          5
     하의         25
dtype: int64

In [30]:
product_분류포함.query("카테고리 == '가방'").groupby(['스타일']).size()

스타일
러블리     1
미니멀    12
빈티지     4
스트릿     8
캐주얼    20
dtype: int64

In [31]:
product_분류포함.query("카테고리 == '슈즈'").groupby(['스타일']).size()

스타일
러블리    12
미니멀     8
빈티지    11
스트릿     4
캐주얼    13
dtype: int64

In [4]:
공유df = pd.read_csv("review_전처리.csv").drop(columns='keyword')
공유df.to_csv("review_전처리.csv", index=False, encoding='utf-8-sig')

/var/folders/4z/77ppy0w97mq12l8j00pcdgr80000gn/T/ipykernel_5231/38691114.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12,13,14,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  공유df = pd.read_csv("review_전처리.csv").drop(columns='keyword')


In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("review_전처리.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154560 entries, 0 to 154559
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   리뷰순서          154560 non-null  object 
 1   카테고리          121792 non-null  object 
 2   브랜드           121792 non-null  object 
 3   상품명           121792 non-null  object 
 4   작성자           121792 non-null  object 
 5   작성 날짜         121792 non-null  object 
 6   옵션            121728 non-null  object 
 7   사이즈           118734 non-null  object 
 8   퀄리티           116640 non-null  object 
 9   색감            114991 non-null  object 
 10  정보            86943 non-null   object 
 11  리뷰 점수         121791 non-null  float64
 12  리뷰 코멘트        116724 non-null  object 
 13  배송형태          113090 non-null  object 
 14  포토리뷰          113090 non-null  object 
 15  상품ID          113090 non-null  float64
 16  review_count  113090 non-null  object 
 17  review_tfidf  113090 non-null  object 
 18  clea

/var/folders/4z/77ppy0w97mq12l8j00pcdgr80000gn/T/ipykernel_6376/2707026465.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12,13,14,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("review_전처리.csv")
